In [2]:
import numpy as np
import pandas as pd

print(f'numpy version: {np.__version__}')
print(f'pandas version: {pd.__version__}')

numpy version: 2.0.0
pandas version: 2.2.2


In [6]:
#in this notebook, im studying different types of merging and aggregations in padas, using a pets dataframe as example
#initializing dataframe

pets = pd.DataFrame(
    data = {
            'name' : ['Mr. Snuggles', 'Honey Chew Chew', 'Professor', 'Chairman Meow', 'Neighbelline'],
            'type' : ['cat', 'dog', 'dog', 'cat', 'horse']
    },
    index = [71, 42, 11, 98, 42],
)
pets.index.rename('pet_id', inplace=True)
print(pets)

                   name   type
pet_id                        
71         Mr. Snuggles    cat
42      Honey Chew Chew    dog
11            Professor    dog
98        Chairman Meow    cat
42         Neighbelline  horse


In [9]:
visits = pd.DataFrame(
    data = {
            'pet_id' : [42, 31, 71, 42, 98, 42],
            'date' : ['2019-03-15', '2019-03-15', '2019-04-05', '2019-04-06', '2019-04-12', '2019-04-12']
    }
)
visits.index.rename('visit_id', inplace=True)

print(visits)

          pet_id        date
visit_id                    
0             42  2019-03-15
1             31  2019-03-15
2             71  2019-04-05
3             42  2019-04-06
4             98  2019-04-12
5             42  2019-04-12


In [17]:
#when the key id has different names across tables, you have to specify each column name
#This causes the resulting table to inherit the right table index 'visit_id'
pd.merge(left=pets, right=visits, how='inner', left_index=True, right_on='pet_id')

#when both table share a common key column 'pet_id', you pass it one time as argument, and the resulting table has its own index values
#since our 'visit_id' was the index column name in the right table, it vanishes from the resulting merge operation
pd.merge(left=pets, right=visits, how='inner', on='pet_id')

,pet_id,name,type,date
0,71,Mr. Snuggles,cat,2019-04-05
1,42,Honey Chew Chew,dog,2019-03-15
2,42,Honey Chew Chew,dog,2019-04-06
3,42,Honey Chew Chew,dog,2019-04-12
4,98,Chairman Meow,cat,2019-04-12
5,42,Neighbelline,horse,2019-03-15
6,42,Neighbelline,horse,2019-04-06
7,42,Neighbelline,horse,2019-04-12


In [19]:
#the result table includes the missing id from previous iteration, the professor, and fills it with null since it doesnt appear on the right table
pd.merge(left=pets, right=visits, how='left', on='pet_id')

,pet_id,name,type,date
0,71,Mr. Snuggles,cat,2019-04-05
1,42,Honey Chew Chew,dog,2019-03-15
2,42,Honey Chew Chew,dog,2019-04-06
3,42,Honey Chew Chew,dog,2019-04-12
4,11,Professor,dog,NaN
5,98,Chairman Meow,cat,2019-04-12
6,42,Neighbelline,horse,2019-03-15
7,42,Neighbelline,horse,2019-04-06
8,42,Neighbelline,horse,2019-04-12


In [22]:
#lastly, we have the outer join, including all keys
pd.merge(left=pets, right=visits, how='outer', on='pet_id', indicator=True)

,pet_id,name,type,date,_merge
0,11,Professor,dog,NaN,left_only
1,31,NaN,NaN,2019-03-15,right_only
2,42,Honey Chew Chew,dog,2019-03-15,both
3,42,Honey Chew Chew,dog,2019-04-06,both
4,42,Honey Chew Chew,dog,2019-04-12,both
5,42,Neighbelline,horse,2019-03-15,both
6,42,Neighbelline,horse,2019-04-06,both
7,42,Neighbelline,horse,2019-04-12,both
8,71,Mr. Snuggles,cat,2019-04-05,both
9,98,Chairman Meow,cat,2019-04-12,both


In [26]:
#how do we person anti-joins? I need to know which which ids from pets table arent included in visits tables. Which pets not in visits
mask = pets.index.isin(visits['pet_id'].unique())
pets.loc[~mask]

,name,type
pet_id,,
11,Professor,dog
